In [ ]:
!pip -qq install torch==1.5.1+cpu torchvision==0.6.1+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
! pip -qq install torch-scatter==latest+cpu -f https://pytorch-geometric.com/whl/torch-1.5.0.html
! pip -qq install torch-sparse==latest+cpu -f https://pytorch-geometric.com/whl/torch-1.5.0.html
! pip -qq install torch-cluster==latest+cpu -f https://pytorch-geometric.com/whl/torch-1.5.0.html
! pip -qq install torch-spline-conv==latest+cpu -f https://pytorch-geometric.com/whl/torch-1.5.0.html
! pip -qq install torch-geometric

In [ ]:
import os
import time
import random
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import torch.nn.functional as F
import torch_geometric.nn as gnn
from torch_geometric.data import Dataset, Data, DataLoader
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

In [ ]:
df = pd.read_csv('../input/cmsnewsamples/new-smaples.csv').drop(columns = 'Unnamed: 0')
df = df.drop(columns = [i for i in df.columns if '_1' in i])
df['non_hits'] = df[[i for i in df.columns if 'mask' in i]].sum(axis=1)
df = df[df['non_hits']==0].reset_index(drop=True)

df['1/pT'] = df['q/pt'].abs()
def label(a):
    if a<=10:
        return 0
    if a>10 and a<=30:
        return 1
    if a>30 and a<=100:
        return 2
    if a>100:
        return 3

df['pT'] = 1/df['1/pT']
    
df['pT_classes'] = df['pT'].apply(label)

features = ['emtf_phi_'+str(i) for i in [0,2,3,4]] + ['emtf_theta_'+str(i) for i in [0,2,3,4]] + ['fr_'+str(i) for i in [0,2,3,4]] + ['old_emtf_phi_'+str(i) for i in [0,2,3,4]]
labels_1 = ['pT']
labels_2 = ['pT_classes']
labels_3 = ['vx']

scaler_1 = StandardScaler()
df[features] = scaler_1.fit_transform(df[features])

scaler_3 = MinMaxScaler()
df[labels_3] = scaler_3.fit_transform(df[labels_3])

In [ ]:
shuffled_list = list(range(len(df)))
random.Random(242).shuffle(shuffled_list)
shuffled_list = np.array_split(np.array(shuffled_list), 10)

In [ ]:
edge_index = torch.tensor([(0,1),(1,2),(2,3),(3,2),(2,1),(1,0)], dtype=torch.long).T
X_data = df[features].to_numpy()
Y_data = df[labels_1].to_numpy()
def process_data(i):
  graph = X_data[i].reshape(-1,4).T
  y = Y_data[i]
  data = Data(x=torch.tensor(graph, dtype=torch.float), y=torch.tensor(y, dtype=torch.float), edge_index=edge_index)
  return data

In [ ]:
class TriggerDataset(Dataset):
    def __init__(self, root, indexes=list(range(len(df))), transform=None, pre_transform=None):
        super(TriggerDataset, self).__init__(root, transform, pre_transform)
        self.indexes = indexes
        self.length = len(self.indexes)

    @property
    def raw_file_names(self):
        return ['vgc']

    @property
    def processed_file_names(self):
        return ['vghv']

    def download(self):
        return None

    def process(self):
        return None

    def len(self):
        return self.length

    def get(self, idx):
        return process_data(self.indexes[idx])

In [ ]:
class MPNN(torch.nn.Module):
    def __init__(self):
      super(MPNN, self).__init__()
      self.conv1 = gnn.SAGEConv(4,64)
#       self.conv2 = gnn.SAGEConv(128,64)
      self.conv3 = gnn.SAGEConv(64,32)
      self.pool1 = gnn.TopKPooling(32, ratio=0.9)
      self.lin1 = torch.nn.Linear(64, 128)
      self.lin2 = torch.nn.Linear(128, 16)
      self.lin3 = torch.nn.Linear(16, 16)
      self.lin4 = torch.nn.Linear(16, 1)

    def forward(self, data):
      x, edge_index, batch = data.x, data.edge_index, data.batch
      x = F.relu(self.conv1(x, edge_index))
#       x = F.relu(self.conv2(x, edge_index))
      x = F.relu(self.conv3(x, edge_index))
      x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
      x = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
      x = F.relu(self.lin1(x))
      x = F.relu(self.lin2(x))
      x = self.lin3(x)
      x = self.lin4(x).squeeze(1)

      return x

In [ ]:
batch_size = 512
epochs = 100

In [ ]:
val_batch, test_batch = 1, 2
train_loader = DataLoader(TriggerDataset('./',np.concatenate([shuffled_list[j] for j in range(10) if j not in (val_batch, test_batch)])), batch_size=batch_size, shuffle=True, num_workers = 4) 
val_loader = DataLoader(TriggerDataset('./',shuffled_list[val_batch]), batch_size=batch_size) 
test_loader = DataLoader(TriggerDataset('./',shuffled_list[test_batch]), batch_size=batch_size)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MPNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=1, factor=0.5)
# mse = torch.nn.MSELoss()

In [ ]:
def mse(outputs, labels):
    weights = torch.tensor(labels<80, dtype=torch.float)*labels + torch.tensor(labels>=80, dtype=torch.float)*torch.tensor(labels<160, dtype=torch.float)*labels*2.4 + torch.tensor(labels>=160, dtype=torch.float)*10
    error = weights*(((outputs-labels)/labels)**2)
    return torch.mean(error)

In [ ]:
m_train_loss = []
m_val_loss = []
m_test_loss = []
min_val_loss = float('inf')
for epoch in range(epochs):
  train_loss = 0
  val_loss = 0
  pbar = tqdm(train_loader)
  for data in pbar:
    data = data.to(device)
    optimizer.zero_grad()
    outputs = model(data)
    labels = data.y
    loss = mse(outputs, labels)
    loss.backward()
    optimizer.step()
    pbar.set_description('MSELoss: '+str(loss.cpu().detach().numpy()))
    train_loss += loss.cpu().detach()/len(train_loader)

  for data in val_loader:
    data = data.to(device)
    optimizer.zero_grad()
    outputs = model(data)
    labels = data.y
    loss = mse(outputs, labels)
    val_loss += loss.cpu().detach()/len(val_loader)
  if val_loss.detach().numpy()<min_val_loss:
    min_val_loss = val_loss.cpu().detach().numpy()
    torch.save(model.state_dict(), 'model.pth')
  lr_scheduler.step(val_loss)
  print('Epoch: ', str(epoch+1)+'/'+str(epochs))
  print('Training MSELoss: ', train_loss.numpy())
  print('Validation MSELoss: ', val_loss.numpy())
  m_train_loss.append(train_loss.numpy())
  m_val_loss.append(val_loss.numpy())
  if epoch>10 and max(m_val_loss[-11:-1])<val_loss.numpy():
    print( max(m_val_loss[-11:-1]),val_loss.numpy())
    break

In [ ]:
model = MPNN().to(device)
model.load_state_dict(torch.load('model.pth'))
test_loss = 0
true = []
preds = []
for data in test_loader:
  data = data.to(device)
  optimizer.zero_grad()
  outputs = model(data)
  labels = data.y
  true += list(labels.detach().numpy())
  preds += list(outputs.detach().numpy())
  loss = mse(outputs, labels)
  test_loss += loss/len(test_loader)
print('Test MSELoss: ', test_loss.detach().numpy())
OOF_preds = pd.DataFrame()
OOF_preds['true_value'] = true
OOF_preds['preds'] = preds
OOF_preds['row'] = shuffled_list[test_batch]
OOF_preds.to_csv('OOF_preds.csv')

In [ ]:
df = pd.read_csv('OOF_preds.csv').drop(columns = ['Unnamed: 0'])
df = df.sort_values(by = 'row').reset_index(drop = True)
df['True_pT'] = df['true_value']
df['Predicted_pT'] = df['preds']

In [ ]:
df

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

MAE1 = []
dx = 0.5
for i in tqdm(range(int(2/dx),int(150/dx))):
    P = df[(df['True_pT']>=(i-1)*dx)&(df['True_pT']<=(i+1)*dx)]
    try:
        p = mae(P['True_pT'],P['Predicted_pT'])
    except:
        p=0
    MAE1.append(p)
MAE1 = MAE1[:146]
plt.plot([i*dx for i in range(int(75/dx))],[0]*int(int(75/dx)-len(MAE1))+MAE1,label = 'FCNN')

In [ ]:
def pT_classes(x):
    if x>=25:
        return 'Above 25 GeV'
    else:
        return 'Below 25 GeV'

print(classification_report(df['True_pT'].apply(pT_classes), df['Predicted_pT'].apply(pT_classes)))